In [1]:
## Install Libraries 
pip install ollama langchain beautifulsoup4 chromadb gradio

Note: you may need to restart the kernel to use updated packages.


In [1]:
import ollama

from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

import requests
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import bs4

import urllib.request

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

import bs4
from urllib.request import urlopen as request
from bs4 import BeautifulSoup as soup
from bs4 import SoupStrainer as strainer

from bs4 import BeautifulSoup as soup
from bs4 import SoupStrainer as strainer


In [3]:
## Import url for the webpage to used

url = ('https://www.consumerfinance.gov/rules-policy/regulations/1024/17/')


In [6]:
## 1a0. - Make a Call to import data from URL

client = request(url)

page_html = client.read()

# Create a SoupStrainer object to only parse p tags
soup_strainer = strainer("p")

from bs4 import BeautifulSoup

# Parse the HTML document using the SoupStrainer object
soup = BeautifulSoup(page_html, "html.parser", parse_only=soup_strainer)

In [9]:
## 1a. convert the web page data into text file

master_data_2 = []

with open(r'mortgage_1.txt', 'w') as fp:
    
    for data in soup.find_all("p"):
        
        extract_text = data.get_text()
        
        master_data_2.append(extract_text)
        
        fp.write("%s\n" % extract_text)

In [10]:
# 1b. Load text data from a file using TextLoader
loader = TextLoader("./mortgage_1.txt")
document = loader.load()

In [11]:
## 1c. Split the text into chunks

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(document)

In [12]:
# 2. Create Ollama embeddings and vector store
embeddings = OllamaEmbeddings(model="mistral")

In [13]:
# 2a. Define vectorstore 
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [14]:
vectorstore

In [15]:
# 3. Call Ollama Llama3 model
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model='llama3', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

In [16]:
# 4. RAG Setup
retriever = vectorstore.as_retriever()

In [17]:
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [18]:
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = combine_docs(retrieved_docs)
    return ollama_llm(question, formatted_context)

In [19]:
# 5. Use the RAG App
result = rag_chain("mortgage loan?")

In [20]:
result

"A mortgage loan refers to a type of loan where the property being purchased or refinanced serves as collateral for the loan. The loan allows the borrower to borrow money from a lender to finance the purchase or refinancing of a home, and the property is pledged as security for the loan.\n\nThe passage does not specifically discuss mortgage loans, but rather provides definitions and guidelines related to escrow accounts and disbursements. It appears that the context is focused on servicers' responsibilities in managing escrow accounts for borrowers with federally related mortgage loans."

In [21]:
print(result)

A mortgage loan refers to a type of loan where the property being purchased or refinanced serves as collateral for the loan. The loan allows the borrower to borrow money from a lender to finance the purchase or refinancing of a home, and the property is pledged as security for the loan.

The passage does not specifically discuss mortgage loans, but rather provides definitions and guidelines related to escrow accounts and disbursements. It appears that the context is focused on servicers' responsibilities in managing escrow accounts for borrowers with federally related mortgage loans.


In [22]:
# 5. Use the RAG App
result_2 = rag_chain("Escrow Account Disclosure?")

In [23]:
# 5. Use the RAG App
result_3 = rag_chain("Annual escrow account statement?")

In [24]:
# 5. Use the RAG App
result_4 = rag_chain("Escrow Account?")

In [25]:
# 5. Use the RAG App
result_5 = rag_chain("Transfer of servicing")

In [26]:
# 5. Use the RAG App
result_6 = rag_chain("shortage or deficiency in escrow account")

In [35]:
# 5. Use the RAG App
result_7 = rag_chain("Submission at settlement, or within 45 calendar days of settlement")

In [36]:
# 5. Use the RAG App
result_8 = rag_chain("Format for initial escrow account statement")

In [37]:
# 5. Use the RAG App
result_9 = rag_chain("No annual statements in the case of default, foreclosure, or bankruptcy")

In [38]:
# 5. Use the RAG App
result_10 = rag_chain("Timely payments")

In [39]:
# 5. Use the RAG App
result_11 = rag_chain("TContents of annual escrow account statement.")


In [27]:
result_2

'Escrow Account Disclosure refers to the requirement for mortgage servicers to provide annual statements to borrowers outlining the status of their escrow accounts. The disclosure must include information such as:\n\n1. The beginning and ending balances of the escrow account\n2. The amounts paid into the escrow account during the year\n3. The amounts disbursed from the escrow account for taxes, insurance premiums, and other charges\n4. Any changes to the escrow account or payment schedule\n5. A history of the account since the last annual statement (if the borrower becomes current after a period of default)\n\nThis disclosure is required by Regulation X of the Real Estate Settlement Procedures Act (RESPA) and is intended to help borrowers understand how their escrow accounts are being managed and to ensure that they are not overpaying or underpaying for property taxes and insurance premiums.'

In [28]:
result_3

'Annual Escrow Account Statement:\n\nThe annual escrow account statement is a required document that must be provided by the servicer to the borrower. The statement must include the following information:\n\n* The beginning and ending balances of the escrow account\n* The total amount of funds in the escrow account during the previous year\n* The disbursements made from the escrow account during the previous year, including the date and amount of each disbursement\n* Any changes to the escrow account, such as the establishment or termination of a discretionary payment\n\nThe servicer must provide this statement to the borrower once a year, typically around the same time each year. The statement is intended to help the borrower understand how their escrow payments are being used and whether they need to make any adjustments to their payments.\n\nExemptions:\n\nThere are two exemptions from the requirement to provide an annual escrow account statement:\n\n* Default or foreclosure: If the

In [29]:
result_4

'An Escrow Account is defined as any account that a servicer establishes or controls on behalf of a borrower to pay taxes, insurance premiums (including flood insurance), or other charges with respect to a federally related mortgage loan. This includes accounts established for this purpose, such as "trust accounts", "reserve accounts", "impound accounts", or other terms used in different localities. An Escrow Account excludes any account that is under the borrower\'s total control.'

In [30]:
result_5

"Transfer of servicing:\n\n* The servicer and borrower can enter into a voluntary agreement for the next escrow accounting year, allowing the borrower to make larger deposits than initially calculated.\n* This agreement is only valid for one year, and a new agreement must be entered into after the next escrow analysis.\n\nShortages (less than 1 month's payment):\n\n* The servicer has three options:\n\t+ Allow the shortage to exist without changing it.\n\t+ Require the borrower to repay the shortage within 30 days.\n\t+ Require the borrower to repay the shortage in equal monthly payments over at least 12 months.\n\nShortages (1 month or more):\n\n* The servicer has two options:\n\t+ Allow the shortage to exist without changing it.\n\t+ Require the borrower to repay the shortage in equal monthly payments over at least 12 months.\n\nDeficiency:\n\n* If a deficiency is found, the servicer can require the borrower to make additional deposits to eliminate the deficiency.\n* If the deficiency

In [40]:
print(result_6)

According to this section, a "shortage" or "deficiency" in an escrow account refers to a situation where the funds held in the escrow account are insufficient to cover the estimated expenses and payments due during the next year.

In other words, if the analysis shows that there is not enough money in the escrow account to meet the expected expenses and payments for the upcoming year, this is considered a shortage or deficiency.


In [41]:
print(result_7)

This text appears to be from Regulation X (RESPA), which is a federal regulation that governs real estate settlement procedures. The specific sections and paragraphs mentioned are related to escrow accounts, deficiencies, and disbursements for property taxes.

Here's a breakdown of the main points:

* Paragraphs (3) and (4): Discuss the requirements for calculating deficiencies in escrow accounts, including notice to borrowers and timing.
* Paragraph (5): Requires servicers to notify borrowers if there is a shortage or deficiency in their escrow account.
* Paragraph (g): Refers to the initial escrow account statement, which must be provided to borrowers within 30 days of settlement or by the next billing cycle.
* Paragraph (l): Discusses discretionary payments, such as credit life insurance, and how they are handled in the escrow account.

The other paragraphs not mentioned specifically are:

* (1) Defines "settlement service" for purposes of this regulation.
* (2) Requires servicers t

In [42]:
print(result_8)

The format for initial escrow account statement according to RESPA Regulation X §10.24.17(i) is as follows:

I. Initial Escrow Account Statement Requirements

* The servicer must provide an initial escrow account statement to the borrower within 45 days of the borrower's initial payment date.
* The statement must include:
	+ The amount of the borrower's monthly payment and how it was calculated.
	+ The estimated annual escrow payments for property taxes, insurance, and other charges.
	+ The estimated annual escrow surplus or shortage.
	+ A description of any escrow account analysis procedures used to determine the initial target balance.

II. Initial Escrow Account Statement Disclosure

* The servicer must disclose to the borrower:
	+ The method used to calculate the initial target balance, including any assumptions made and the source of any data used.
	+ Any changes in the estimated annual escrow payments or the estimated annual escrow surplus or shortage since the last annual statem

In [43]:
print(result_9)

The answer is:

No annual statements are required in the case of default, foreclosure, or bankruptcy.


In [44]:
print(result_10)


Timely payments refer to the payment of mortgage insurance premiums and other escrow account funds in accordance with the terms of the federally related mortgage loan and applicable federal or state laws.

In this context, timely payments are:

* Payments made by the borrower on a regular schedule, such as monthly or annually, as part of their mortgage obligations.
* Payments that are not delinquent or past due, meaning they are made before the due date or within the allowed grace period.

The regulation emphasizes the importance of timely payments in ensuring that the escrow account is properly funded and that surplus funds are handled according to the requirements outlined in the RESPA regulations.


In [45]:
print(result_11)


Based on the provided text, here are some key points regarding the contents of annual escrow account statements:

1. Annual statement exemption: The servicer does not need to submit an annual escrow account statement if the borrower is more than 30 days overdue or in a situation where foreclosure proceedings have begun.
2. Statement content: The statement should include:
	* A computation of the current escrow account balance and whether it is sufficient or insufficient
	* Information on any shortage or surplus
	* Any changes to the escrow account since the previous statement
3. Additional information: If a discretionary payment is made, it should be noted on the initial and annual statements.
4. Timing: The servicer must provide an annual escrow account statement within 30 days of conducting the escrow analysis.

The provided text also defines key terms related to escrow accounts, such as:

1. Shortage: An amount by which a current escrow account balance falls short of the target balan

In [31]:
print(result_2)

Escrow Account Disclosure refers to the requirement for mortgage servicers to provide annual statements to borrowers outlining the status of their escrow accounts. The disclosure must include information such as:

1. The beginning and ending balances of the escrow account
2. The amounts paid into the escrow account during the year
3. The amounts disbursed from the escrow account for taxes, insurance premiums, and other charges
4. Any changes to the escrow account or payment schedule
5. A history of the account since the last annual statement (if the borrower becomes current after a period of default)

This disclosure is required by Regulation X of the Real Estate Settlement Procedures Act (RESPA) and is intended to help borrowers understand how their escrow accounts are being managed and to ensure that they are not overpaying or underpaying for property taxes and insurance premiums.


In [32]:
print(result_3)

Annual Escrow Account Statement:

The annual escrow account statement is a required document that must be provided by the servicer to the borrower. The statement must include the following information:

* The beginning and ending balances of the escrow account
* The total amount of funds in the escrow account during the previous year
* The disbursements made from the escrow account during the previous year, including the date and amount of each disbursement
* Any changes to the escrow account, such as the establishment or termination of a discretionary payment

The servicer must provide this statement to the borrower once a year, typically around the same time each year. The statement is intended to help the borrower understand how their escrow payments are being used and whether they need to make any adjustments to their payments.

Exemptions:

There are two exemptions from the requirement to provide an annual escrow account statement:

* Default or foreclosure: If the borrower defaul

In [33]:
print(result_4)

An Escrow Account is defined as any account that a servicer establishes or controls on behalf of a borrower to pay taxes, insurance premiums (including flood insurance), or other charges with respect to a federally related mortgage loan. This includes accounts established for this purpose, such as "trust accounts", "reserve accounts", "impound accounts", or other terms used in different localities. An Escrow Account excludes any account that is under the borrower's total control.


In [34]:
print(result_5)

Transfer of servicing:

* The servicer and borrower can enter into a voluntary agreement for the next escrow accounting year, allowing the borrower to make larger deposits than initially calculated.
* This agreement is only valid for one year, and a new agreement must be entered into after the next escrow analysis.

Shortages (less than 1 month's payment):

* The servicer has three options:
	+ Allow the shortage to exist without changing it.
	+ Require the borrower to repay the shortage within 30 days.
	+ Require the borrower to repay the shortage in equal monthly payments over at least 12 months.

Shortages (1 month or more):

* The servicer has two options:
	+ Allow the shortage to exist without changing it.
	+ Require the borrower to repay the shortage in equal monthly payments over at least 12 months.

Deficiency:

* If a deficiency is found, the servicer can require the borrower to make additional deposits to eliminate the deficiency.
* If the deficiency is less than one month's p